In [1]:
import bifacial_radiance
import os
import pandas as pd

In [19]:
inifile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\ini_highAzimuth.ini'
weatherfile = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\724666TYA.csv'
testfolder = r'C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Modelchain'

In [14]:
(Params)= bifacial_radiance.load.readconfigurationinputfile(inifile=inifile)

Loading times to restrict weather data to


In [15]:
Params[0]

{'weatherFile': '724666TYA.CSV',
 'getEPW': False,
 'simulationname': '_test_high_azimuth_angle_modelchain',
 'moduletype': 'test-module',
 'rewriteModule': True,
 'cellLevelModule': False,
 'axisofrotationTorqueTube': False,
 'torqueTube': False,
 'hpc': False,
 'tracking': False,
 'cumulativeSky': False,
 'selectTimes': True,
 'latitude': '37.5',
 'longitude': '-77.6'}

In [16]:
Params[2]

{'starttime': '06_17_13', 'endtime': '06_17_13'}

In [24]:
Params[0]['testfolder'] = testfolder
Params[0]['weatherFile'] = weatherfile
Params[2].update({'starttime': '06_17_13', 'endtime':'06_17_14'}); 

In [25]:
demo2, analysis = bifacial_radiance.modelchain.runModelChain(*Params ) 



New bifacial_radiance simulation starting. 
Version:  0.4.0-dev2+15.geae1033
path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Modelchain
Loading times to restrict weather data to
Reading weather file C:\Users\sayala\Documents\GitHub\bifacial_radiance\tests\724666TYA.csv
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.800 avg
1 nonzero albedo values.
Available module names: ['PrismSolar-Bi60', 'basic-module', 'test-module']

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten


Using Pre-determined Module Type: test-module 
Creating ~2

In [37]:
demo2.trackerdict

dict_keys(['2021-06-17_1300', '2021-06-17_1400'])

In [39]:
#CEC Module
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.
modfilter2 = db.index.str.startswith('Pr') & db.index.str.endswith('BHC72-400')
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

1  modules selected. Name of 1st entry:  Prism Solar Technologies_ Inc. BHC72-400


In [47]:
trackerdict = demo2.calculatePerformanceModule(CECMod)

Bifaciality factor of module stored is  0.9


In [46]:
demo2.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict


,Unnamed: 0,dhi,ghi,Wm2Back,Wm2Front,theta,surf_tilt,surf_azm,clearance_height,effective_irradiance,Pout_module,Wm2BackAvg,Wm2FrontAvg,BifiRatio
0,2021-06-17_1300,172,959,"[150.38653333333332, 112.35766666666666, 102.3...","[895.5752333333334, 895.5537333333333, 895.531...",10.0,10.0,30.0,0.2,1069.774363,386.325463,190.280542,898.521875,0.190594
1,2021-06-17_1400,306,702,"[114.94426666666668, 93.72626666666666, 87.733...","[643.6093, 644.4763, 645.3445666666668, 647.03...",10.0,10.0,30.0,0.2,784.296925,291.653189,153.390079,646.245854,0.213620


In [ ]:
#assert np.round(np.mean(analysis.backRatio),2) == 0.20  # bifi ratio was == 0.22 in v0.2.2
assert np.mean(analysis.Wm2Front) == pytest.approx(898, rel = 0.005)  # was 912 in v0.2.3
assert np.mean(analysis.Wm2Back) == pytest.approx(189, rel = 0.02)  # was 182 in v0.2.2